In [ ]:
#Worked in COLAB, try using TPU for faster processing
#Spam Detection Model using CNN
import tensorflow as tf
tf.__version__

'2.3.0'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv('SMSSpamCollection', sep="\t", names = ['label','message'])

In [ ]:
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 133kB/s 
     |████████████████████████████████| 983kB 50.5MB/s 
     |████████████████████████████████| 266kB 52.2MB/s 
     |████████████████████████████████| 1.3MB 35.3MB/s 
     |████████████████████████████████| 471kB 49.4MB/s 
     |████████████████████████████████| 1.1MB 49.5MB/s 
     |████████████████████████████████| 2.9MB 52.7MB/s 
     |████████████████████████████████| 890kB 49.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.0-cp36-none-any.whl size=25274500 sha256=017870250552a0e01e375a3ff4d83bbb557ae7a3060bd07c3c6b255425b9d63f
  Stored in directory: /root/.cache/pip/wheels/5c/73/05/f36d0027bb6575384e21506dbba8db36a7825f15a24f09b2d5
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=1223b2b80cb6964c4c26d239478a492b8007138a309973420306595bcabb5462
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [ ]:
import ktrain
from ktrain import text

In [ ]:
#Binarize labels
data['label']=data['label'].map({'ham':0, 'spam':1})
data.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data.shape

(5572, 2)

In [ ]:
#Seperate data as features and label
features = data.message.values
label = data.label.values

In [ ]:
#Train test split
from sklearn.model_selection import train_test_split
X_Train,X_Test,y_Train,y_Test = train_test_split(features,
                                                 label,
                                                 test_size=0.2,
                                                 random_state = 20)

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_array(x_train=X_Train,y_train=y_Train,x_test=X_Test,y_test=y_Test,
                                                                   maxlen=189, class_names=['0','1'],
                                                                   preprocess_mode='bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [ ]:
model = text.text_classifier(name='bert', 
                             train_data=(X_train, y_train), preproc=preproc)


Is Multi-Label? False
maxlen is 189
done.


In [ ]:
# wrap model and data in ktrain.Learner object
learner = ktrain.get_learner(model=model, train_data=(X_train,y_train),
                             val_data=(X_test,y_test),
                             batch_size=2)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
2229/2229 [==============================] - 649s 291ms/step - loss: 0.0762 - accuracy: 0.9686 - val_loss: 0.0493 - val_accuracy: 0.9892


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
data = ['You have won a Lottery',
        'Pay your School fees',
        'Mom tried reaching you',
        "You have won a trip to Malaysia"]

In [ ]:
predictor.predict(data)

['0', '0', '0', '0']

In [ ]:
predictor.get_classes()

['0', '1']

In [ ]:
predictor.save('/content/bert_SPAMHAMsms')

In [ ]:
predictor_load = ktrain.load_predictor('/content/bert_SPAMHAMsms')

In [ ]:
predictor_load.get_classes()

['0', '1']

In [ ]:
predictor_load.predict(data)

['0', '0', '0', '0']